In [5]:
#include <iostream>
using namespace std;
int a = 2;
%r cout << a << endl;
string b  = "Hello";
%r cout << b << endl;
void f(int i, const string & s){
    while(i--)
        cout << i << ": " << s << endl;
}

2
Hello


In [7]:
void g(void){
    f(a, b);
}

In [8]:
%r g();                                                                     
%r a = 5;                                                                       
%r f(a, "World");  

4: Hello
3: Hello
2: Hello
1: Hello
0: Hello
4: World
3: World
2: World
1: World
0: World


In [4]:
long a = 10;
%r cout << a << endl;
%r f(a, "AAAA");

10
9: AAAA
8: AAAA
7: AAAA
6: AAAA
5: AAAA
4: AAAA
3: AAAA
2: AAAA
1: AAAA
0: AAAA


In [ ]:
 int c = 43;
%r cout << c << "dsfdsf";

In [ ]:
%r cout << 32 << endl;